In [1]:
# Dependencies
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv("../db/nutrition.csv")
# df = pd.read_csv("nutrition.csv", index_col ="NDB_No")

In [3]:
df.head()

NDB_No               Shrt_Desc  Water  Energy  Protein  Lipid_Total  \
0    1001        BUTTER,WITH SALT  15.87     717     0.85        81.11   
1    1002  BUTTER,WHIPPED,W/ SALT  16.72     718     0.49        78.30   
2    1003    BUTTER OIL,ANHYDROUS   0.24     876     0.28        99.48   
3    1004             CHEESE,BLUE  42.41     353    21.40        28.74   
4    1005            CHEESE,BRICK  41.11     371    23.24        29.68   

   Carbohydrate  Fiber  Sugar_Total  Calcium  ...  Vitamin_K  FA_Sat_g  \
0          0.06    0.0         0.06       24  ...        7.0    51.368   
1          2.87    0.0         0.06       23  ...        4.6    45.390   
2          0.00    0.0         0.00        4  ...        8.6    61.924   
3          2.34    0.0         0.50      528  ...        2.4    18.669   
4          2.79    0.0         0.51      674  ...        2.5    18.764   

   FA_Mono_g  FA_Poly_g  Cholestrol  Weight_grams                 Weight_desc  \
0     21.021      3.043         215          5.00  1 pat,  (1" sq, 1/3" high)   
1     19.874      3.331         225          3.80  1 pat,  (1" sq, 1/3" high)   
2     28.732      3.694         256         12.80                      1 tbsp   
3      7.778      0.800          75         28.35                        1 oz   
4      8.598      0.784          94        132.00                1 cup, diced   

   GmWt_2       GmWt_Desc2  Refuse_Pct  
0    14.2           1 tbsp           0  
1     9.4           1 tbsp           0  
2   205.0            1 cup           0  
3    17.0     1 cubic inch           0  
4   113.0  1 cup, shredded           0  

[5 rows x 52 columns]

In [4]:
# df.loc[1001]
# df.loc[1129]

In [5]:
df.columns

Index(['NDB_No', 'Shrt_Desc', 'Water', 'Energy', 'Protein', 'Lipid_Total',
       'Carbohydrate', 'Fiber', 'Sugar_Total', 'Calcium', 'Iron', 'Magnesium',
       'Phosphorus', 'Potassium', 'Sodium', 'Zinc', 'Copper', 'Manganese',
       'Selenium', 'Vitamin_C', 'Thiamin', 'Riboflavin', 'Niacin',
       'Panto_Acid', 'Vitamin_B6', 'Folate_Total', 'Folic_Acid',
       'Food_Folate_mcg', 'Folate_DFE_mcg', 'Choline_Tot_mg', 'Vitamin_B12',
       'Vit_A_IU', 'Vitamin_A', 'Retinol', 'Alpha_Carot_mcg', 'Beta_Carot_mcg',
       'Beta_Crypt_mcg', 'Lycopene_mcg', 'Lut_Zea_mcg', 'Vitamin_E',
       'Vitamin_D', 'Vit_D_IU', 'Vitamin_K', 'FA_Sat_g', 'FA_Mono_g',
       'FA_Poly_g', 'Cholestrol', 'Weight_grams', 'Weight_desc', 'GmWt_2',
       'GmWt_Desc2', 'Refuse_Pct'],
      dtype='object')

In [6]:
# food_data = data.set_index('Shrt_Desc').T.to_dict()
#food_data.keys()
#X = [v for k,v in df.items()]
# food_data
#food_data.keys()
# item_list = [i for i in food_data.keys()]
# item_list[0]

# Change the columns to per calorie value

In [7]:
df["Protein/cal"] = df["Protein"] / df["Energy"]
df["Carbohydrtes/cal"] = df["Carbohydrate"] / df["Energy"]
df["Sodium/cal"] = df["Sodium"]/ df["Energy"]
df["Cholestrol/cal"] = df["Cholestrol"]/ df["Energy"]
df["Sugar/cal"] = df["Sugar_Total"]/ df["Energy"]
df["Calcium/cal"] = df["Calcium"]/ df["Energy"]

In [8]:
df_percalorie = df[["Shrt_Desc", "Protein/cal", "Carbohydrtes/cal", "Sodium/cal", "Cholestrol/cal", "Sugar/cal", "Calcium/cal"]]

In [9]:
#df_percalorie = df[["Shrt_Desc", 'Protein_g', "Protein/cal"]]

In [10]:
df_percalorie.head()

Shrt_Desc  Protein/cal  Carbohydrtes/cal  Sodium/cal  \
0        BUTTER,WITH SALT     0.001185          0.000084    0.896792   
1  BUTTER,WHIPPED,W/ SALT     0.000682          0.003997    0.811978   
2    BUTTER OIL,ANHYDROUS     0.000320          0.000000    0.002283   
3             CHEESE,BLUE     0.060623          0.006629    3.246459   
4            CHEESE,BRICK     0.062642          0.007520    1.509434   

   Cholestrol/cal  Sugar/cal  Calcium/cal  
0        0.299861   0.000084     0.033473  
1        0.313370   0.000084     0.032033  
2        0.292237   0.000000     0.004566  
3        0.212465   0.001416     1.495751  
4        0.253369   0.001375     1.816712

## Removing null values from dataframe

In [11]:
df_percalorie.isnull().sum()

Shrt_Desc            0
Protein/cal         36
Carbohydrtes/cal    31
Sodium/cal           9
Cholestrol/cal      39
Sugar/cal           38
Calcium/cal         13
dtype: int64

In [12]:
df_percalorie = df_percalorie.dropna(how='any',axis=0)

In [13]:
df_percalorie.isnull().sum()

Shrt_Desc           0
Protein/cal         0
Carbohydrtes/cal    0
Sodium/cal          0
Cholestrol/cal      0
Sugar/cal           0
Calcium/cal         0
dtype: int64

In [14]:
df_percalorie.shape

(8751, 7)

## Cosine similarity

In [15]:
df_percalorie.columns

Index(['Shrt_Desc', 'Protein/cal', 'Carbohydrtes/cal', 'Sodium/cal',
       'Cholestrol/cal', 'Sugar/cal', 'Calcium/cal'],
      dtype='object')

In [16]:
X = df_percalorie[['Protein/cal', 'Carbohydrtes/cal', 'Sodium/cal', 'Cholestrol/cal', 'Sugar/cal', 'Calcium/cal']].values

In [17]:
# X = df_percalorie[['Protein/cal']].values

In [18]:
X_norm = Normalizer().fit_transform(X)

In [19]:
# eggs
# df_percalorie.loc[1129]

In [20]:
# Find the index for search item
print(int(df_percalorie[df_percalorie['Shrt_Desc']=='SOY PROT ISOLATE,K TYPE'].index.values))

5009


In [21]:
# X_norm[5009][0]

In [22]:
# food with max protein values
# df_sort = df_percalorie.sort_values("Protein/cal", ascending = False)
# df_sort.head(10)

In [23]:
# df_percalorie.loc[16422]

In [24]:
# df_percalorie
# NDB_No

## Display list of 10 similar items

In [25]:
# df_percalorie

In [26]:
# X_norm[16422][0]
# dfb = int(df_percalorie[df_percalorie['NDB_No']==16422].index[0])
# dfb
# dfbb = int(df[df['A']==8].index[0])
print(int(df_percalorie[df_percalorie['Shrt_Desc']=='SOY PROT ISOLATE,K TYPE'].index.values))

5009


In [27]:
# df.index[df['BoolCol']].tolist()

In [28]:
# df_sort.head(10)

In [ ]:
X_term = cv.transform(["choclte chip sookies"])

In [29]:
similarities = cosine_similarity(X_norm[5009].reshape(1,-1), X_norm)
k = 10
result = np.sort(np.argpartition(similarities[0], len(similarities[0]) - k)[-k:])

#idxmax = np.arange(start=simularities[0], stop = simularities[5] , step=1)
df_percalorie.iloc[result]

Shrt_Desc  Protein/cal  \
5021       LOMA LINDA TENDER ROUNDS W/ GRAVY,CND,UNPREP     0.112414   
5025                  LOMA LINDA TENDER BITS,CND,UNPREP     0.111852   
5032            WORTHINGTON LOFAT VEJA-LINKS,CND,UNPREP     0.128455   
5034                WORTHINGTON PRIME STAKES,CND,UNPREP     0.075556   
5040                  WORTHINGTON VEJA-LINKS,CND,UNPREP     0.094194   
5043     WORTHINGTON MEATLESS CORNED BF ROLL,FRZ,UNPREP     0.075918   
5046               WORTHINGTON PROSAGE LINKS,FRZ,UNPREP     0.141259   
5047                WORTHINGTON PROSAGE ROLL,FRZ,UNPREP     0.075096   
5050                   WORTHINGTON STRIPPLES,FRZ,UNPREP     0.035838   
5054  MORNINGSTAR FARMS BRKFST SAUSAGE LINKS,FRZ,UNPREP     0.120755   

      Carbohydrtes/cal  Sodium/cal  Cholestrol/cal  Sugar/cal  Calcium/cal  
5021          0.051034    3.055172        0.006897   0.006897     0.096552  
5025          0.061481    4.540741        0.000000   0.004444     0.125926  
5032          0.034959    4.975610        0.016260   0.005691     0.162602  
5034          0.055556    3.555556        0.007407   0.002222     0.111111  
5040          0.026452    3.419355        0.019355   0.003226     0.148387  
5043          0.040408    3.093878        0.004082   0.008571     0.097959  
5046          0.034965    5.727273        0.013986   0.005594     0.181818  
5047          0.022989    2.555556        0.007663   0.000766     0.107280  
5050          0.041329    4.228324        0.005780   0.002312     0.121387  
5054          0.042767    4.213836        0.012579   0.006289     0.163522

In [30]:
np.sort(np.argpartition(similarities[0], len(similarities[0]) - k)[-k:])

array([4984, 4988, 4995, 4997, 5003, 5006, 5009, 5010, 5013, 5017],
      dtype=int32)

In [31]:
result

array([4984, 4988, 4995, 4997, 5003, 5006, 5009, 5010, 5013, 5017],
      dtype=int32)

In [32]:
similarities[0]

array([0.94893456, 0.93361018, 0.01076693, ..., 0.49341898, 0.8034373 ,
       0.49374694])

In [ ]:
# Recommend 5 items similar to text in search string
def similar_items(term):
    X_term = cv.transform([term])
    simularities = cosine_similarity(X_term, X)
    k = 10
    result = np.sort(np.argpartition(simularities[0], len(simularities[0]) - k)[-k:])

    #idxmax = np.arange(start=simularities[0], stop = simularities[5] , step=1)
    return df.loc[result]
if __name__ == "__main__":
    term = "egg"
    term1 = "choclate cooies mnt"
    print(advanced_search(term)["Shrt_Desc"])

In [35]:
# df = pd.DataFrame({
#     "eggs": [1, 2, 3, 5],
#     "bacon": [12, 123, 3, 5],
#     "spam": [12, 23, 4, 5]
# })
# # index 0 eggs, 1 bacon, 2 spam
# X_norm = Normalize().fit_transform(X)
# cosine_simularity(X_norm[0], X_norm[~0])
# X_norm[~0].dot(X_norm[0].T)
# X_sim = X_norm.dot(X_norm.T)
# [
#     [.123, 1, 1],
#     [.240, 1, *],
#     [1, *, *]
# ]
# sim_eggs = X_sim[0]
# # search for the index that is closest but not the egg index